In [251]:
import pandas as pd
import numpy as np
import os

In [2]:
os.listdir()

['.Rhistory',
 '.DS_Store',
 'data_processed',
 '02_summary_loop.R',
 'Untitled.ipynb',
 '02_summary_tables.R',
 'export_result_oct21.xlsx',
 '.RData',
 '.ipynb_checkpoints',
 'data_raw',
 '01_load_clean.R']

In [3]:
os.listdir("/Users/saeed/Library/CloudStorage/OneDrive-DalhousieUniversity/0- Dalhousie University - Saeed/CPHSC/basic docs/Food is Health/Hospital Food/working/data_raw")

['~$HospitalFoodEmissions_Dec2023-Nov2024.xlsx',
 'Food_ImpactFactors(ONmodel).xlsx',
 'HospitalFoodEmissions_Dec2023-Nov2024.xlsx',
 'Food_ImpactFactors(ONmodel)_updated_Backward.xlsx']

In [4]:
def clean_names(df):
    first = list(df.columns)
    second = list()
    for name in first:
        name2 = str(name).lower()
        if name2[-1]==" ":
            name2 = name2[:-1]
        name2 = name2.replace(" ", "_")
        second.append(name2)
    df.columns=second

# 00 - ON model

In [5]:
# original
#onmodel = pd.read_excel("data_raw/Food_ImpactFactors(ONmodel).xlsx")
#onmodel.columns = onmodel.iloc[3]
#onmodel = onmodel.iloc[4:, 1:]
#onmodel.reset_index(drop=True, inplace=True)
#clean_names(onmodel)
#print('Shape:', onmodel.shape)
#onmodel.head()

In [184]:
# updated
onmodel_u = pd.read_excel("data_raw/Food_ImpactFactors(ONmodel)_updated_Backward.xlsx")
print('Shape:', onmodel_u.shape)
onmodel_u.head()

Shape: (105, 2)


,food_items,gwp_of_1_kg_food
0,S_chocolate_HH_consumed_ON,15.83009
1,S_icecream_consumed_ON,4.90982
2,S_sugar_consumed_ON,1.65425
3,S_tortillachips_consumed_ON,3.85381
4,S_ice_fruit_consumed_ON,1.30562


In [6]:
# animal included (15 items)
animal_list = ["A_beef_consumed_ON",
"A_mixed_meat_consumed_ON",
"A_butter_consumed_ON",
"A_pork_consumed_ON",
"A_cheese_consumed_ON",
"A_salmon_consumed_ON",
"A_chicken_consumed_ON",
"Turkey_Proxy_Chicken_Consumed",
"Tuna_Bianchietal2022",
"S_icecream_consumed_ON",
"A_egg_consumed_ON",
"Haddock_Bianchietal2022",
"Cod_Bianchietal2022",
"A_yogurt_consumed_ON",
"A_milk_consumed_ON"]

# 01 - Dec 23

In [7]:
#dec23 = pd.read_excel("data_raw/HospitalFoodEmissions_Dec2023-Nov2024.xlsx")
#clean_names(dec23)
#merge2 = dec23.iloc[:78].merge(onmodel, on='food_items', how='left')
#merge2.loc[1,'gwp_of_1_kg_food_x'] = (4.28+2.6)/2
#new_model = merge2[merge2['gwp_of_1_kg_food_y'].isna()]
#new_model = new_model[['food_items', 'gwp_of_1_kg_food_x']]
#new_model.columns = onmodel.columns
#new_model
#pd.concat([onmodel,new_model])
#onmodel_update = pd.concat([onmodel,new_model])
#onmodel_update.reset_index(drop=True, inplace=True)
##onmodel_update.to_excel("data_raw/Food_ImpactFactors(ONmodel)_updated_Backward.xlsx", index=False)
#onmodel_update.head()

In [7]:
dec23 = pd.read_excel("data_raw/HospitalFoodEmissions_Dec2023-Nov2024.xlsx")
clean_names(dec23)
dec23 = dec23.iloc[:,0:3]
dec23 = pd.concat([dec23.iloc[:78], dec23.iloc[81:124]], axis = 0)
dec23

,food_items,quantity,sum_of_portionunitstandard_g
0,A_beef_consumed_ON,16098,1444621.5
1,Vegetables_MontegoBlend,8502,638964
2,A_mixed_meat_consumed_ON,701,43586
3,A_butter_consumed_ON,17511,59515.5
4,Pizza_Flatbread_Cheese_Flatbred_Tomato,721,53314
...,...,...,...
119,Mayonnaise Ind RTS,2455,22682
120,Salad Roasted Vegetable,3268,373285
121,Topping Whipped,1429,17760
122,Thickner,458,3799


In [8]:
def gen_table(month, onmodel_u):
    m_dec23 = month.merge(onmodel_u, on='food_items', how='left')
    
    table_1 = m_dec23[~m_dec23['gwp_of_1_kg_food'].isna()]
    #table_1.loc[:,['gwp_of_1_kg_food']] = table_1['gwp_of_1_kg_food'].round(2)
    table_1.loc[:,["gwp_x_kg"]] = (table_1['sum_of_portionunitstandard_g'] / 1000 * table_1['gwp_of_1_kg_food'])#.astype(float).round(2)
    
    table_2 = m_dec23[m_dec23['gwp_of_1_kg_food'].isna()].dropna(axis=1)
    table_2.loc[:,["sum_of_portionunitstandard_kg"]] = table_2['sum_of_portionunitstandard_g'] / 1000
    
    animal = table_1[table_1['food_items'].isin(animal_list)]
    
    animal_sec = animal.copy()
    animal_sec.loc[animal_sec[animal_sec['food_items'] == "A_beef_consumed_ON"].index, 'gwp_of_1_kg_food'] = animal_sec.loc[animal_sec[animal_sec['food_items'] == "A_chicken_consumed_ON"].index, 'gwp_of_1_kg_food'].values
    #animal_sec.loc[:,["gwp_x_kg"]] = (animal_sec['sum_of_portionunitstandard_g'] / 1000 * animal_sec['gwp_of_1_kg_food'])#.astype(float).round(2)
    animal_sec.loc[animal_sec[animal_sec['food_items'] == "A_beef_consumed_ON"].index, 'gwp_x_kg'] = animal_sec.loc[animal_sec[animal_sec['food_items'] == "A_chicken_consumed_ON"].index, 'gwp_of_1_kg_food'].values * animal_sec.loc[animal_sec[animal_sec['food_items'] == "A_beef_consumed_ON"].index, 'sum_of_portionunitstandard_g'].values / 1000
    
    return table_1, table_2, animal, animal_sec

In [9]:
table_1, table_2, animal, animal_sec = gen_table(dec23, onmodel_u)

In [8]:
m_dec23 = dec23.merge(onmodel_u, on='food_items', how='left')
m_dec23

,food_items,quantity,sum_of_portionunitstandard_g,gwp_of_1_kg_food
0,A_beef_consumed_ON,16098,1444621.5,55.64676
1,Vegetables_MontegoBlend,8502,638964,3.44000
2,A_mixed_meat_consumed_ON,701,43586,33.62227
3,A_butter_consumed_ON,17511,59515.5,19.79185
4,Pizza_Flatbread_Cheese_Flatbred_Tomato,721,53314,15.17000
...,...,...,...,...
116,Mayonnaise Ind RTS,2455,22682,NaN
117,Salad Roasted Vegetable,3268,373285,NaN
118,Topping Whipped,1429,17760,NaN
119,Thickner,458,3799,NaN


In [9]:
table_1 = m_dec23[~m_dec23['gwp_of_1_kg_food'].isna()]
#table_1.loc[:,['gwp_of_1_kg_food']] = table_1['gwp_of_1_kg_food'].round(2)
table_1.loc[:,["gwp_x_kg"]] = (table_1['sum_of_portionunitstandard_g'] / 1000 * table_1['gwp_of_1_kg_food'])#.astype(float).round(2)
table_1

,food_items,quantity,sum_of_portionunitstandard_g,gwp_of_1_kg_food,gwp_x_kg
0,A_beef_consumed_ON,16098,1444621.5,55.64676,80388.505901
1,Vegetables_MontegoBlend,8502,638964,3.44000,2198.03616
2,A_mixed_meat_consumed_ON,701,43586,33.62227,1465.46026
3,A_butter_consumed_ON,17511,59515.5,19.79185,1177.921849
4,Pizza_Flatbread_Cheese_Flatbred_Tomato,721,53314,15.17000,808.77338
...,...,...,...,...,...
73,Supp_Isosource+HN_MilkProtein_SoyProtein_Water...,614,145190,13.33000,1935.3827
74,B_tea_brewed_consumed_ON,39279,9774750,0.03785,369.974288
75,Greenbeans_Proxy_chickpeas,5288,307206,0.92000,282.62952
76,FrenchToast_Proxy_Eggs_Bread,6164,184956.2,6.12000,1131.931944


In [11]:
table_1.sum()

food_items                      A_beef_consumed_ONVegetables_MontegoBlendA_mix...
quantity                                                                   866964
sum_of_portionunitstandard_g                                          71329475.34
gwp_of_1_kg_food                                                        412.94109
gwp_x_kg                                                            261396.715328
dtype: object

In [90]:
table_1.sum()

food_items                      A_beef_consumed_ONVegetables_MontegoBlendA_mix...
quantity                                                                   866964
sum_of_portionunitstandard_g                                          71329475.34
gwp_of_1_kg_food                                                        412.94109
gwp_x_kg                                                            261396.715328
dtype: object

In [10]:
table_2 = m_dec23[m_dec23['gwp_of_1_kg_food'].isna()].dropna(axis=1)
table_2.loc[:,["sum_of_portionunitstandard_kg"]] = table_2['sum_of_portionunitstandard_g'] / 1000
table_2.head()

,food_items,quantity,sum_of_portionunitstandard_g,sum_of_portionunitstandard_kg
78,Syrup Pancake Ind RTS,6165,58496,58.496
79,Syrup Pancake Low Calorie Ind RTS,454,4800,4.8
80,Spread FruitRite Ind RTS,8,75,0.075
81,Soup Broth Chicken Bone HM,475,108391,108.391
82,Soup Broth Vegetable HM,105,24593,24.593


In [12]:
table_2.sum()

food_items                       Syrup Pancake Ind RTSSyrup Pancake Low Calorie...
quantity                                                                    148632
sum_of_portionunitstandard_g                                            9012437.72
sum_of_portionunitstandard_kg                                           9012.43772
dtype: object

In [92]:
table_2.sum()

food_items                       Syrup Pancake Ind RTSSyrup Pancake Low Calorie...
quantity                                                                    148632
sum_of_portionunitstandard_g                                            9012437.72
sum_of_portionunitstandard_kg                                           9012.43772
dtype: object

In [11]:
animal = table_1[table_1['food_items'].isin(animal_list)]
animal

,food_items,quantity,sum_of_portionunitstandard_g,gwp_of_1_kg_food,gwp_x_kg
0,A_beef_consumed_ON,16098,1444621.5,55.64676,80388.505901
2,A_mixed_meat_consumed_ON,701,43586,33.62227,1465.46026
3,A_butter_consumed_ON,17511,59515.5,19.79185,1177.921849
8,A_pork_consumed_ON,1664,259183,11.14398,2888.330168
9,A_cheese_consumed_ON,31753,707440,11.13663,7878.497527
11,A_salmon_consumed_ON,149,14700,9.05449,133.101003
13,A_chicken_consumed_ON,17596,949876.7,8.15745,7748.571686
14,Turkey_Proxy_Chicken_Consumed,7818,538187.5,8.16000,4391.61
16,Tuna_Bianchietal2022,1746,103020,7.50000,772.65
20,S_icecream_consumed_ON,18320,2004795,4.90982,9843.182587


In [65]:
#animal.sort_values(by='sum_of_portionunitstandard_g')

In [12]:
animal.sum()

food_items                      A_beef_consumed_ONA_mixed_meat_consumed_ONA_bu...
quantity                                                                   312518
sum_of_portionunitstandard_g                                           21572380.4
gwp_of_1_kg_food                                                        185.61432
gwp_x_kg                                                            155640.955788
dtype: object

In [13]:
animal_sec = animal.copy()
animal_sec.loc[animal_sec[animal_sec['food_items'] == "A_beef_consumed_ON"].index, 'gwp_of_1_kg_food'] = animal_sec.loc[animal_sec[animal_sec['food_items'] == "A_chicken_consumed_ON"].index, 'gwp_of_1_kg_food'].values
#animal_sec.loc[:,["gwp_x_kg"]] = (animal_sec['sum_of_portionunitstandard_g'] / 1000 * animal_sec['gwp_of_1_kg_food'])#.astype(float).round(2)
animal_sec.loc[animal_sec[animal_sec['food_items'] == "A_beef_consumed_ON"].index, 'gwp_x_kg'] = animal_sec.loc[animal_sec[animal_sec['food_items'] == "A_chicken_consumed_ON"].index, 'gwp_of_1_kg_food'].values * animal_sec.loc[animal_sec[animal_sec['food_items'] == "A_beef_consumed_ON"].index, 'sum_of_portionunitstandard_g'].values / 1000

animal_sec

,food_items,quantity,sum_of_portionunitstandard_g,gwp_of_1_kg_food,gwp_x_kg
0,A_beef_consumed_ON,16098,1444621.5,8.15745,11784.427655
2,A_mixed_meat_consumed_ON,701,43586,33.62227,1465.46026
3,A_butter_consumed_ON,17511,59515.5,19.79185,1177.921849
8,A_pork_consumed_ON,1664,259183,11.14398,2888.330168
9,A_cheese_consumed_ON,31753,707440,11.13663,7878.497527
11,A_salmon_consumed_ON,149,14700,9.05449,133.101003
13,A_chicken_consumed_ON,17596,949876.7,8.15745,7748.571686
14,Turkey_Proxy_Chicken_Consumed,7818,538187.5,8.16000,4391.61
16,Tuna_Bianchietal2022,1746,103020,7.50000,772.65
20,S_icecream_consumed_ON,18320,2004795,4.90982,9843.182587


In [151]:
animal_sec.sum()

food_items                      A_beef_consumed_ONA_mixed_meat_consumed_ONA_bu...
quantity                                                                   312518
sum_of_portionunitstandard_g                                           21572380.4
gwp_of_1_kg_food                                                        138.12501
gwp_x_kg                                                             87036.877541
dtype: object

## Summary 1 & 2

In [54]:
animal_kg = animal['sum_of_portionunitstandard_g'].sum() / 1000
animal_gwp = animal['gwp_x_kg'].sum()
print('Animal, KG:', animal_kg)
print('Animal, GWP:', animal_gwp)

Animal, KG: 21572.3804
Animal, GWP: 155640.95578755002


In [44]:
n_animal_w_GWP = table_1[~table_1['food_items'].isin(animal_list)]
n_animal_w_GWP_kg = n_animal_w_GWP['sum_of_portionunitstandard_g'].sum() / 1000
n_animal_w_GWP_gwp = n_animal_w_GWP['gwp_x_kg'].sum()

print('Not Animal with GWP, KG:', n_animal_w_GWP_kg)
print('Not Animal with GWP, GWP:', n_animal_w_GWP_gwp)

Not Animal with GWP, KG: 49757.094939999995
Not Animal with GWP, GWP: 105755.75954019597


In [39]:
n_animal_wh_GWP_kg = table_2['sum_of_portionunitstandard_g'].sum() / 1000
print('Not Animal without GWP, KG:', n_animal_wh_GWP_kg)

Not Animal without GWP, KG: 9012.437719999993


In [45]:
non_animal_kg = n_animal_w_GWP_kg + n_animal_wh_GWP_kg
print('100% Not Animal, KG:', non_animal_kg)

100% Not Animal, KG: 58769.53265999999


In [46]:
unestimated_coef = n_animal_wh_GWP_kg / n_animal_w_GWP_kg
print("Unestimated coreffient:", unestimated_coef)

Unestimated coreffient: 0.1811286959350765


In [47]:
n_animal_wh_GWP_gwp_estimate = unestimated_coef * n_animal_w_GWP_gwp
print('Not Animal without GWP, GWP:', n_animal_wh_GWP_gwp_estimate)

Not Animal without GWP, GWP: 19155.402813139222


In [48]:
non_animal_gwp = n_animal_w_GWP_gwp + n_animal_wh_GWP_gwp_estimate 
print('100% Not Animal, GWP:', non_animal_gwp)

100% Not Animal, GWP: 124911.1623533352


In [60]:
summary = pd.DataFrame(columns=['mass_kg', 'percentage_kg', 'gwp_total', 'percentage_gwp'],
                      index=['Animal prodcuts on Menu', 'Everything else on menu w/ estimates for GWP',
                            ' Unestimated Food Mass','Total'])

summary['mass_kg'] = [animal_kg, n_animal_w_GWP_kg, n_animal_wh_GWP_kg, (animal_kg+ n_animal_w_GWP_kg+ n_animal_wh_GWP_kg)]
summary['gwp_total'] = [animal_gwp, n_animal_w_GWP_gwp, n_animal_wh_GWP_gwp_estimate, (animal_gwp+ n_animal_w_GWP_gwp+ n_animal_wh_GWP_gwp_estimate)]

summary['percentage_kg'] = (summary['mass_kg'] / summary.loc['Total','mass_kg'] * 100).round(2)
summary['percentage_gwp'] = (summary['gwp_total'] / summary.loc['Total','gwp_total'] * 100).round(2)

summary

,mass_kg,percentage_kg,gwp_total,percentage_gwp
Animal prodcuts on Menu,21572.38040,26.85,155640.955788,55.48
Everything else on menu w/ estimates for GWP,49757.09494,61.93,105755.759540,37.70
Unestimated Food Mass,9012.43772,11.22,19155.402813,6.83
Total,80341.91306,100.00,280552.118141,100.00


In [15]:
def summary_1(table_1, table_2, animal):
    animal_kg = animal['sum_of_portionunitstandard_g'].sum() / 1000
    animal_gwp = animal['gwp_x_kg'].sum()

    n_animal_w_GWP = table_1[~table_1['food_items'].isin(animal_list)]
    n_animal_w_GWP_kg = n_animal_w_GWP['sum_of_portionunitstandard_g'].sum() / 1000
    n_animal_w_GWP_gwp = n_animal_w_GWP['gwp_x_kg'].sum()
    
    n_animal_wh_GWP_kg = table_2['sum_of_portionunitstandard_g'].sum() / 1000
    
    non_animal_kg = n_animal_w_GWP_kg + n_animal_wh_GWP_kg
    
    unestimated_coef = n_animal_wh_GWP_kg / n_animal_w_GWP_kg

    n_animal_wh_GWP_gwp_estimate = unestimated_coef * n_animal_w_GWP_gwp
    
    non_animal_gwp = n_animal_w_GWP_gwp + n_animal_wh_GWP_gwp_estimate 

    summary = pd.DataFrame(columns=['mass_kg', 'percentage_kg', 'gwp_total', 'percentage_gwp'],
                      index=['Animal prodcuts on Menu', 'Everything else on menu w/ estimates for GWP',
                            ' Unestimated Food Mass','Total'])

    summary['mass_kg'] = [animal_kg, n_animal_w_GWP_kg, n_animal_wh_GWP_kg, (animal_kg+ n_animal_w_GWP_kg+ n_animal_wh_GWP_kg)]
    summary['gwp_total'] = [animal_gwp, n_animal_w_GWP_gwp, n_animal_wh_GWP_gwp_estimate, (animal_gwp+ n_animal_w_GWP_gwp+ n_animal_wh_GWP_gwp_estimate)]
    
    summary['percentage_kg'] = (summary['mass_kg'] / summary.loc['Total','mass_kg'] * 100).round(2)
    summary['percentage_gwp'] = (summary['gwp_total'] / summary.loc['Total','gwp_total'] * 100).round(2)
    
    return summary

In [16]:
summary_1(table_1, table_2, animal)

,mass_kg,percentage_kg,gwp_total,percentage_gwp
Animal prodcuts on Menu,21572.38040,26.85,155640.955788,55.48
Everything else on menu w/ estimates for GWP,49757.09494,61.93,105755.759540,37.70
Unestimated Food Mass,9012.43772,11.22,19155.402813,6.83
Total,80341.91306,100.00,280552.118141,100.00


In [30]:
ss = summary_1(table_1, table_2, animal_sec)
ss

,variables,mass_kg,percentage_kg,gwp_total,percentage_gwp
0,Animal prodcuts on Menu,21572.38040,26.85,87036.877541,41.07
1,Everything else on menu w/ estimates for GWP,49757.09494,61.93,105755.759540,49.90
2,Unestimated Food Mass,9012.43772,11.22,19155.402813,9.04
3,Total,80341.91306,100.00,211948.039895,100.00


# month and summary with function

In [7]:
def gen_table(month, onmodel_u):
    m_dec23 = month.merge(onmodel_u, on='food_items', how='left')
    
    table_1 = m_dec23[~m_dec23['gwp_of_1_kg_food'].isna()]
    #table_1.loc[:,['gwp_of_1_kg_food']] = table_1['gwp_of_1_kg_food'].round(2)
    table_1.loc[:,["gwp_x_kg"]] = (table_1['sum_of_portionunitstandard_g'] / 1000 * table_1['gwp_of_1_kg_food'])#.astype(float).round(2)
    
    table_2 = m_dec23[m_dec23['gwp_of_1_kg_food'].isna()].dropna(axis=1)
    table_2.loc[:,["sum_of_portionunitstandard_kg"]] = table_2['sum_of_portionunitstandard_g'] / 1000
    
    animal = table_1[table_1['food_items'].isin(animal_list)]
    
    animal_sec = animal.copy()
    animal_sec.loc[animal_sec[animal_sec['food_items'] == "A_beef_consumed_ON"].index, 'gwp_of_1_kg_food'] = animal_sec.loc[animal_sec[animal_sec['food_items'] == "A_chicken_consumed_ON"].index, 'gwp_of_1_kg_food'].values
    #animal_sec.loc[:,["gwp_x_kg"]] = (animal_sec['sum_of_portionunitstandard_g'] / 1000 * animal_sec['gwp_of_1_kg_food'])#.astype(float).round(2)
    animal_sec.loc[animal_sec[animal_sec['food_items'] == "A_beef_consumed_ON"].index, 'gwp_x_kg'] = animal_sec.loc[animal_sec[animal_sec['food_items'] == "A_chicken_consumed_ON"].index, 'gwp_of_1_kg_food'].values * animal_sec.loc[animal_sec[animal_sec['food_items'] == "A_beef_consumed_ON"].index, 'sum_of_portionunitstandard_g'].values / 1000
    
    return table_1, table_2, animal, animal_sec

In [8]:
def summary_1(table_1, table_2, animal):
    animal_kg = animal['sum_of_portionunitstandard_g'].sum() / 1000
    animal_gwp = animal['gwp_x_kg'].sum()

    n_animal_w_GWP = table_1[~table_1['food_items'].isin(animal_list)]
    n_animal_w_GWP_kg = n_animal_w_GWP['sum_of_portionunitstandard_g'].sum() / 1000
    n_animal_w_GWP_gwp = n_animal_w_GWP['gwp_x_kg'].sum()
    
    n_animal_wh_GWP_kg = table_2['sum_of_portionunitstandard_g'].sum() / 1000
    
    non_animal_kg = n_animal_w_GWP_kg + n_animal_wh_GWP_kg
    
    unestimated_coef = n_animal_wh_GWP_kg / n_animal_w_GWP_kg

    n_animal_wh_GWP_gwp_estimate = unestimated_coef * n_animal_w_GWP_gwp
    
    non_animal_gwp = n_animal_w_GWP_gwp + n_animal_wh_GWP_gwp_estimate 

    summary = pd.DataFrame(columns=['mass_kg', 'percentage_kg', 'gwp_total', 'percentage_gwp'],
                      index=['Animal prodcuts on Menu', 'Everything else on menu w/ estimates for GWP',
                            ' Unestimated Food Mass','Total'])

    summary['mass_kg'] = [animal_kg, n_animal_w_GWP_kg, n_animal_wh_GWP_kg, (animal_kg+ n_animal_w_GWP_kg+ n_animal_wh_GWP_kg)]
    summary['gwp_total'] = [animal_gwp, n_animal_w_GWP_gwp, n_animal_wh_GWP_gwp_estimate, (animal_gwp+ n_animal_w_GWP_gwp+ n_animal_wh_GWP_gwp_estimate)]
    
    summary['percentage_kg'] = (summary['mass_kg'] / summary.loc['Total','mass_kg'] * 100).round(2)
    summary['percentage_gwp'] = (summary['gwp_total'] / summary.loc['Total','gwp_total'] * 100).round(2)
    
    summary['variables'] = summary.index
    summary = summary[['variables', 'mass_kg', 'percentage_kg', 'gwp_total', 'percentage_gwp']]
    summary.reset_index(drop=True, inplace=True)
    return summary

In [31]:
table_1, table_2, animal, animal_sec = gen_table(dec23, onmodel_u)
summary_original = summary_1(table_1, table_2, animal)
summary_second = summary_1(table_1, table_2, animal_sec)

In [9]:
def save_tables_to_excel(tables, sheet_name, file_name, row_gap=2, col_gap=2):
    """
    Layout:
        Row 1: Table1 | Table3 | Table5
        Row 2: Table2 | Table4 | Table6

    Second row starts 2 rows below the table above it in each column.
    If the file exists and the sheet exists, we overlay writes on that sheet.
    """
    if len(tables) != 6:
        raise ValueError("You must provide exactly 6 DataFrames.")

    # Column blocks sized by the widest table so spacing is consistent
    max_cols = max(df.shape[1] for df in tables)
    col_positions = [i * (max_cols + col_gap) for i in range(3)]

    top_idx = [0, 2, 4]
    bot_idx = [1, 3, 5]

    # Writer setup: overlay on existing sheet instead of erroring or creating extra sheets
    if os.path.exists(file_name):
        writer = pd.ExcelWriter(file_name, engine="openpyxl", mode="a", if_sheet_exists="overlay")
    else:
        writer = pd.ExcelWriter(file_name, engine="openpyxl")

    # Top row
    bottom_starts = []
    for i, idx in enumerate(top_idx):
        df = tables[idx]
        startcol = col_positions[i]
        startrow = 0
        df.to_excel(
            writer, sheet_name=sheet_name,
            startrow=startrow, startcol=startcol,
            index=False, header=True
        )
        bottom_starts.append(df.shape[0] + 1 + row_gap)  # +1 for header row

    # Bottom row
    for i, idx in enumerate(bot_idx):
        df = tables[idx]
        startcol = col_positions[i]
        startrow = bottom_starts[i]
        df.to_excel(
            writer, sheet_name=sheet_name,
            startrow=startrow, startcol=startcol,
            index=False, header=True
        )

    writer.close()


In [64]:
save_tables_to_excel([table_1, table_2, animal, summary_original, animal_sec, summary_second],
                     sheet_name='Dec23', file_name=r'export_result_oct21.xlsx')

# Final function

In [10]:
def process_month(month, onmodel_u, sheet_name, file_name, animal_list):

    # Generate base tables
    table_1, table_2, animal, animal_sec = gen_table(month, onmodel_u)

    # Summaries
    summary_original = summary_1(table_1, table_2, animal)
    summary_second   = summary_1(table_1, table_2, animal_sec)

    # Collect for saving
    tables = [table_1, table_2, animal, summary_original, animal_sec, summary_second]

    # Save
    save_tables_to_excel(tables, sheet_name=sheet_name, file_name=file_name)
    
    print("Month", month, 'saved successfully!')
    #return tables  # optionally return them if you want to use them later


In [70]:
#process_month(
#    dec23, onmodel_u,
#    sheet_name="Nov23",
#    file_name=r"export_result_oct21.xlsx",
#    animal_list=animal_list
#)
#

## Jan 24

In [11]:
#xls = pd.ExcelFile("data_raw/HospitalFoodEmissions_Dec2023-Nov2024.xlsx").sheet_names
xls_sheet_name = pd.ExcelFile("data_raw/HospitalFoodEmissions_Dec2023-Nov2024.xlsx").sheet_names
print(xls_sheet_name)

["Dec '23 GWP Data", "Jan '24 GWP", "Feb '24 GWP Data", "March '24 GWP Data", 'April 2024', 'May 2024', 'June 2024', "July '24 GWP Data", 'August 2024', 'September 2024', 'October 2024 Gwp Data', 'November 2024', 'Combined Data', 'Figure 4', 'Figure 1', 'Figure 2', 'Beef Monthly Masses']


In [13]:
xls_sheet_name[1]

"Jan '24 GWP"

In [29]:
#jan24.iloc[80:132]

In [71]:
jan24 = pd.read_excel("data_raw/HospitalFoodEmissions_Dec2023-Nov2024.xlsx", sheet_name=xls_sheet_name[1])
clean_names(jan24)
jan24 = jan24.iloc[:,0:3]
jan24 = pd.concat([jan24.iloc[:77], jan24.iloc[80:132]], axis = 0)
jan24

,food_items,quantity,sum_of_portionunitstandard_g
0,A_beef_consumed_ON,17178,1521302.5
1,Vegetables_MontegoBlend,7986,598644
2,A_mixed_meat_consumed_ON,820,44536
3,A_butter_consumed_ON,19011,202725
4,Pizza_Flatbread_Cheese_Flatbred_Tomato,556,40890
...,...,...,...
127,Spread FruitRite Ind RTS,10,250
128,Syrup Pancake Ind RTS,5691,57536
129,Syrup Pancake Low Calorie Ind RTS,625,6432
130,Thickener SimplyThk Moderat Honey IndRTS,283,1980


In [68]:
#process_month(
#    jan24, onmodel_u,
#    sheet_name="Jan24",
#    file_name=r"export_result_oct21.xlsx",
#    animal_list=animal_list
#)
#

## Feb 24

In [69]:
xls_sheet_name[2]

"Feb '24 GWP Data"

In [136]:
feb24 = pd.read_excel("data_raw/HospitalFoodEmissions_Dec2023-Nov2024.xlsx", sheet_name=xls_sheet_name[2])
clean_names(feb24)
feb24 = feb24.iloc[:,0:3]
f = feb24.iloc[81:140].copy()
f.loc[:,'sum_of_portionunitstandard_g'] = f['quantity']
f.loc[:,'quantity']= np.nan
feb24 = pd.concat([feb24.iloc[:76], f], axis = 0)
feb24

,food_items,quantity,sum_of_portionunitstandard_g
0,A_beef_consumed_ON,16434,1457266.5
1,Vegetables_MontegoBlend,8349,617824
2,A_mixed_meat_consumed_ON,879,32946
3,A_butter_consumed_ON,19318,65505
4,Pizza_Flatbread_Cheese_Flatbred_Tomato,580,42194
...,...,...,...
135,Juice Cranberry Mildly Thick Level 2 Ind RTS,NaN,1062
136,Sauce Soy Portion Ind RTS,NaN,621
137,Salad Garden Side with Vegan Mozzarella (House...,NaN,500
138,Spread FruitRite Ind RTS,NaN,75


In [138]:
table_1, table_2, animal, animal_sec = gen_table(feb24, onmodel_u)

#process_month(
#    feb24, onmodel_u,
#    sheet_name="Feb24",
#    file_name=r"export_result_oct21.xlsx",
#    animal_list=animal_list
#)
#

## Mar 24

In [148]:
xls_sheet_name[3]

"March '24 GWP Data"

In [168]:
mar24 = pd.read_excel("data_raw/HospitalFoodEmissions_Dec2023-Nov2024.xlsx", sheet_name=xls_sheet_name[3])
clean_names(mar24)
mar24 = mar24.iloc[:,0:3]
m = mar24.iloc[79:125].copy()
m.loc[:,'sum_of_portionunitstandard_g'] = m['quantity']
m.loc[:,'quantity']= np.nan
mar24 = pd.concat([mar24.iloc[:75], m], axis = 0)
mar24

,food_items,quantity,sum_of_portionunitstandard_g
0,Vegetables_MontegoBlend,8600,631076
1,A_beef_consumed_ON,15565,1352187.5
2,A_mixed_meat_consumed_ON,1097,45144
3,A_butter_consumed_ON,20934,69360.5
4,Pizza_Flatbread_Cheese_Flatbred_Tomato,7270,201488
...,...,...,...
120,Condiment Mustard Yellow Bulk,NaN,680
121,Milk Coconut 17/19% Cnd,NaN,120
122,Prunes Puree RTS,NaN,76
123,TF MCT Oil - On Additional Request List,NaN,75


In [193]:
#set(mar24.iloc[:75].food_items) - set(onmodel_u.food_items)
table_1, table_2, animal, animal_sec = gen_table(mar24, onmodel_u)

#process_month(
#    mar24, onmodel_u,
#    sheet_name="Mar24",
#    file_name=r"export_result_oct21.xlsx",
#    animal_list=animal_list
#)
#

Month                                   food_items quantity  \
0                    Vegetables_MontegoBlend     8600   
1                         A_beef_consumed_ON    15565   
2                   A_mixed_meat_consumed_ON     1097   
3                       A_butter_consumed_ON    20934   
4     Pizza_Flatbread_Cheese_Flatbred_Tomato     7270   
..                                       ...      ...   
120            Condiment Mustard Yellow Bulk      NaN   
121                  Milk Coconut 17/19% Cnd      NaN   
122                         Prunes Puree RTS      NaN   
123  TF MCT Oil - On Additional Request List      NaN   
124                 Spread FruitRite Ind RTS      NaN   

    sum_of_portionunitstandard_g  
0                         631076  
1                      1352187.5  
2                          45144  
3                        69360.5  
4                         201488  
..                           ...  
120                          680  
121                          

## Apr 24

In [195]:
xls_sheet_name[4]

'April 2024'

In [205]:
apr24 = pd.read_excel("data_raw/HospitalFoodEmissions_Dec2023-Nov2024.xlsx", sheet_name=xls_sheet_name[4])
clean_names(apr24)
apr24 = apr24.iloc[:,0:3]
apr24 = pd.concat([apr24.iloc[:76], apr24.iloc[81:129]], axis = 0)
apr24

,food_items,quantity,sum_of_portionunitstandard_g
0,A_beef_consumed_ON,16542,1650330.5
1,Vegetables_MontegoBlend,7464,560688
2,A_mixed_meat_consumed_ON,1071,49780
3,A_butter_consumed_ON,18843,62650.5
4,Pizza_Flatbread_Cheese_Flatbred_Tomato,443,32918
...,...,...,...
124,Spread FruitRite Ind RTS,3,50
125,Syrup Pancake Ind RTS,5505,53456
126,Syrup Pancake Low Calorie Ind RTS,603,5888
127,Thickener SimplyThk Moderat Honey IndRTS,823,8868


In [217]:
#set(mar24.iloc[:75].food_items) - set(onmodel_u.food_items)
table_1, table_2, animal, animal_sec = gen_table(apr24, onmodel_u)

#process_month(
#    apr24, onmodel_u,
#    sheet_name="Apr24",
#    file_name=r"export_result_oct21.xlsx",
#    animal_list=animal_list
#)
#

In [215]:
animal_sec.sum()

food_items                      A_beef_consumed_ONA_mixed_meat_consumed_ONA_bu...
quantity                                                                   329078
sum_of_portionunitstandard_g                                           23640246.9
gwp_of_1_kg_food                                                        138.12501
gwp_x_kg                                                             92690.360956
dtype: object

## May 24

In [218]:
xls_sheet_name[5]

'May 2024'

In [244]:
may24 = pd.read_excel("data_raw/HospitalFoodEmissions_Dec2023-Nov2024.xlsx", sheet_name=xls_sheet_name[5])
clean_names(may24)
may24 = may24.iloc[:,0:3]
may24 = pd.concat([may24.iloc[:76], may24.iloc[80:123]], axis = 0)
may24

,food_items,quantity,sum_of_portionunitstandard_g
0,A_beef_consumed_ON,16154,1405370.5
1,Vegetables_MontegoBlend,7375,541200
2,A_mixed_meat_consumed_ON,1069,38836
3,A_butter_consumed_ON,20181,68508
4,Pizza_Flatbread_Cheese_Flatbred_Tomato,306,22216
...,...,...,...
118,Spread FruitRite Ind RTS,18,450
119,Syrup Pancake Ind RTS,6127,56560
120,Syrup Pancake Low Calorie Ind RTS,728,5824
121,Thickener SimplyThk Moderat Honey IndRTS,278,2508


In [250]:
#set(mar24.iloc[:75].food_items) - set(onmodel_u.food_items)
table_1, table_2, animal, animal_sec = gen_table(may24, onmodel_u)

#process_month(
#    may24, onmodel_u,
#    sheet_name="May24",
#    file_name=r"export_result_oct21.xlsx",
#    animal_list=animal_list
#)
#

In [248]:
animal_sec.sum()

food_items                      A_beef_consumed_ONA_mixed_meat_consumed_ONA_bu...
quantity                                                                   325350
sum_of_portionunitstandard_g                                           21173690.5
gwp_of_1_kg_food                                                        138.12501
gwp_x_kg                                                             86504.387883
dtype: object

## Jun 24

In [252]:
xls_sheet_name[6]

'June 2024'

In [264]:
jun24 = pd.read_excel("data_raw/HospitalFoodEmissions_Dec2023-Nov2024.xlsx", sheet_name=xls_sheet_name[6])
clean_names(jun24)
jun24 = jun24.iloc[:,0:3]
jun24 = pd.concat([jun24.iloc[:75], jun24.iloc[80:127]], axis = 0)
jun24

,food_items,quantity,sum_of_portionunitstandard_g
0,A_beef_consumed_ON,15346,1331837.5
1,Vegetables_MontegoBlend,8034,590136
2,A_mixed_meat_consumed_ON,801,26106
3,A_butter_consumed_ON,18258,63827.5
4,Pizza_Flatbread_Cheese_Flatbred_Tomato,286,21888
...,...,...,...
122,Spread FruitRite Ind RTS,9,150
123,Syrup Pancake Ind RTS,6159,55696
124,Syrup Pancake Low Calorie Ind RTS,448,4672
125,Thickener SimplyThk Moderat Honey IndRTS,435,3552


In [272]:
#set(mar24.iloc[:75].food_items) - set(onmodel_u.food_items)
table_1, table_2, animal, animal_sec = gen_table(jun24, onmodel_u)

#process_month(
#    jun24, onmodel_u,
#    sheet_name="Jun24",
#    file_name=r"export_result_oct21.xlsx",
#    animal_list=animal_list
#)
#

In [270]:
animal_sec.sum()

food_items                      A_beef_consumed_ONA_mixed_meat_consumed_ONA_bu...
quantity                                                                   326675
sum_of_portionunitstandard_g                                           21933476.0
gwp_of_1_kg_food                                                        138.12501
gwp_x_kg                                                             92666.061784
dtype: object

## Jul 24

In [273]:
xls_sheet_name[7]

"July '24 GWP Data"

In [284]:
jul24 = pd.read_excel("data_raw/HospitalFoodEmissions_Dec2023-Nov2024.xlsx", sheet_name=xls_sheet_name[7])
clean_names(jul24)
jul24 = jul24.iloc[:,0:3]
j = jul24.iloc[80:124].copy()
j.loc[:,'sum_of_portionunitstandard_g'] = j['quantity']
j.loc[:,'quantity']= np.nan
jul24 = pd.concat([jul24.iloc[:76], j], axis = 0)
jul24

,food_items,quantity,sum_of_portionunitstandard_g
0,Vegetables_MontegoBlend,7040,519480
1,A_beef_consumed_ON,15872,1427518.5
2,A_mixed_meat_consumed_ON,1067,38190
3,A_butter_consumed_ON,18636,63222.5
4,Pizza_Flatbread_Cheese_Flatbred_Tomato,293,21024
...,...,...,...
119,Juice Apple Tetra Ind RTS,NaN,400
120,Juice Apple Mildly Thick Level 2 Ind RTS,NaN,354
121,Spread FruitRite Ind RTS,NaN,100
122,Gravy Beef LS GF,NaN,62


In [292]:
#set(mar24.iloc[:75].food_items) - set(onmodel_u.food_items)
table_1, table_2, animal, animal_sec = gen_table(jul24, onmodel_u)

#process_month(
#    jul24, onmodel_u,
#    sheet_name="Jul24",
#    file_name=r"export_result_oct21.xlsx",
#    animal_list=animal_list
#)
#

In [290]:
animal_sec.sum()

food_items                      A_beef_consumed_ONA_mixed_meat_consumed_ONA_bu...
quantity                                                                   331183
sum_of_portionunitstandard_g                                           21536271.7
gwp_of_1_kg_food                                                        138.12501
gwp_x_kg                                                             88417.950079
dtype: object

## Aug 24

In [293]:
xls_sheet_name[8]

'August 2024'

In [303]:
aug24 = pd.read_excel("data_raw/HospitalFoodEmissions_Dec2023-Nov2024.xlsx", sheet_name=xls_sheet_name[8])
clean_names(aug24)
aug24 = aug24.iloc[:,0:3]
aug24 = pd.concat([aug24.iloc[:76], aug24.iloc[81:122]], axis = 0)
aug24

,food_items,quantity,sum_of_portionunitstandard_g
0,A_beef_consumed_ON,15546,1911408
1,Vegetables_MontegoBlend,7657,558394
2,A_mixed_meat_consumed_ON,969,36328
3,A_butter_consumed_ON,19317,67292.5
4,Pizza_Flatbread_Cheese_Flatbred_Tomato,407,29376
...,...,...,...
117,Spread FruitRite Ind RTS,1,25
118,Syrup Pancake Ind RTS,6035,55712
119,Syrup Pancake Low Calorie Ind RTS,487,5120
120,Thickener SimplyThk Moderat Honey IndRTS,152,1308


In [310]:
#set(mar24.iloc[:75].food_items) - set(onmodel_u.food_items)
table_1, table_2, animal, animal_sec = gen_table(aug24, onmodel_u)

#process_month(
#    aug24, onmodel_u,
#    sheet_name="Aug24",
#    file_name=r"export_result_oct21.xlsx",
#    animal_list=animal_list
#)
#

In [308]:
animal_sec.sum()

food_items                      A_beef_consumed_ONA_mixed_meat_consumed_ONA_bu...
quantity                                                                   333082
sum_of_portionunitstandard_g                                           20573166.3
gwp_of_1_kg_food                                                        138.12501
gwp_x_kg                                                              85685.98845
dtype: object

## Sep 24

In [311]:
xls_sheet_name[9]

'September 2024'

In [324]:
sep24 = pd.read_excel("data_raw/HospitalFoodEmissions_Dec2023-Nov2024.xlsx", sheet_name=xls_sheet_name[9])
clean_names(sep24)
sep24 = sep24.iloc[:,0:3]
sep24 = pd.concat([sep24.iloc[:76], sep24.iloc[80:115]], axis = 0)
sep24

,food_items,quantity,sum_of_portionunitstandard_g
0,A_beef_consumed_ON,15381,1783496
1,Vegetables_MontegoBlend,7519,581172
2,A_mixed_meat_consumed_ON,1208,41154
3,A_butter_consumed_ON,16960,59323
4,Pizza_Flatbread_Cheese_Flatbred_Tomato,331,36670
...,...,...,...
110,Sauce Teriyaki Thick,679,28455
111,Spread FruitRite Ind RTS,16,350
112,Thickener Simply Thick Mildly Nectar Ind RTS,16,72
113,Thickener SimplyThk Moderat Honey IndRTS,198,1272


In [331]:
#set(mar24.iloc[:75].food_items) - set(onmodel_u.food_items)
table_1, table_2, animal, animal_sec = gen_table(sep24, onmodel_u)

#process_month(
#    sep24, onmodel_u,
#    sheet_name="Sep24",
#    file_name=r"export_result_oct21.xlsx",
#    animal_list=animal_list
#)
#

In [329]:
animal_sec.sum()

food_items                      A_beef_consumed_ONA_mixed_meat_consumed_ONA_bu...
quantity                                                                   324120
sum_of_portionunitstandard_g                                           21498387.9
gwp_of_1_kg_food                                                        138.12501
gwp_x_kg                                                             89009.669526
dtype: object

## Oct 24

In [332]:
xls_sheet_name[10]

'October 2024 Gwp Data'

In [390]:
oct24 = pd.read_excel("data_raw/HospitalFoodEmissions_Dec2023-Nov2024.xlsx", sheet_name=xls_sheet_name[10])
clean_names(oct24)
oct24 = oct24.iloc[:,0:3]
#oct24.columns = sep24.columns
o = oct24.iloc[79:122].copy()
o.loc[:,'sum_of_portionunitstandard_g'] = o['quantity']
o.loc[:,'quantity']= np.nan
oct24 = pd.concat([oct24.iloc[:75], o], axis = 0)
oct24

,food_items,quantity,sum_of_portionunitstandard_g
0,Vegetables_MontegoBlend,7870,569388
1,A_beef_consumed_ON,15960,1433134.5
2,A_mixed_meat_consumed_ON,854,33592
3,A_butter_consumed_ON,18100,62221.5
4,Pizza_Flatbread_Cheese_Flatbred_Tomato,10221,301206
...,...,...,...
117,Syrup Pancake Low Calorie Ind RTS,NaN,3504
118,TF MCT Oil - On Additional Request List,NaN,215
119,Thickener Simply Thick Mildly Nectar Ind RTS,NaN,73
120,Thickener SimplyThk Moderat Honey IndRTS,NaN,2436


In [403]:
#set(mar24.iloc[:75].food_items) - set(onmodel_u.food_items)
table_1, table_2, animal, animal_sec = gen_table(oct24, onmodel_u)

#process_month(
#    oct24, onmodel_u,
#    sheet_name="Oct24",
#    file_name=r"export_result_oct21.xlsx",
#    animal_list=animal_list
#)
#

In [400]:
animal_sec.sum()

food_items                      A_beef_consumed_ONA_mixed_meat_consumed_ONA_bu...
quantity                                                                   331142
sum_of_portionunitstandard_g                                           21258950.5
gwp_of_1_kg_food                                                        138.12501
gwp_x_kg                                                             85821.625042
dtype: object

## Nov 24

In [404]:
xls_sheet_name[11]

'November 2024'

In [416]:
nov24 = pd.read_excel("data_raw/HospitalFoodEmissions_Dec2023-Nov2024.xlsx", sheet_name=xls_sheet_name[11])
clean_names(nov24)
nov24 = nov24.iloc[:,0:3]
nov24 = pd.concat([nov24.iloc[:76], nov24.iloc[80:123]], axis = 0)
nov24

,food_items,quantity,sum_of_portionunitstandard_g
0,A_beef_consumed_ON,15726,1811448
1,Vegetables_MontegoBlend,8597,664976
2,A_mixed_meat_consumed_ON,624,30020
3,A_butter_consumed_ON,18017,58712.5
4,Pizza_Flatbread_Cheese_Flatbred_Tomato,330,48900
...,...,...,...
118,Spice Oregano Leaves,2181,2142
119,Syrup Pancake Ind RTS,5639,51008
120,Syrup Pancake Low Calorie Ind RTS,673,6256
121,Thickener SimplyThk Moderat Honey IndRTS,134,1032


In [424]:
#set(mar24.iloc[:75].food_items) - set(onmodel_u.food_items)
table_1, table_2, animal, animal_sec = gen_table(nov24, onmodel_u)

#process_month(
#    nov24, onmodel_u,
#    sheet_name="Nov24",
#    file_name=r"export_result_oct21.xlsx",
#    animal_list=animal_list
#)
#

In [422]:
animal_sec.sum()

food_items                      A_beef_consumed_ONA_mixed_meat_consumed_ONA_bu...
quantity                                                                   327935
sum_of_portionunitstandard_g                                           21786476.1
gwp_of_1_kg_food                                                        138.12501
gwp_x_kg                                                             90567.799432
dtype: object